In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [3]:
# These are how we are going to transform the images
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [4]:
# load my datasets
train_dataset = datasets.ImageFolder(root='train', transform=transform)
test_dataset = datasets.ImageFolder(root='test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
# Create the model to process images
class EmotionClassifier(nn.Module):
    def __init__(self):
        super(EmotionClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 6 * 6, 256)
        self.fc2 = nn.Linear(256, 7)
        self.dropout = nn.Dropout(0.25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout(x)
        x = x.view(-1, 128 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = EmotionClassifier().to(device)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
# Train model
num_epochs = 250
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}')

Epoch 1/250, Loss: 0.5718
Epoch 2/250, Loss: 0.5603
Epoch 3/250, Loss: 0.5505
Epoch 4/250, Loss: 0.5384
Epoch 5/250, Loss: 0.5245
Epoch 6/250, Loss: 0.5324
Epoch 7/250, Loss: 0.5199
Epoch 8/250, Loss: 0.5060
Epoch 9/250, Loss: 0.4978
Epoch 10/250, Loss: 0.4990
Epoch 11/250, Loss: 0.4846
Epoch 12/250, Loss: 0.4770
Epoch 13/250, Loss: 0.4756
Epoch 14/250, Loss: 0.4657
Epoch 15/250, Loss: 0.4606
Epoch 16/250, Loss: 0.4609
Epoch 17/250, Loss: 0.4546
Epoch 18/250, Loss: 0.4491
Epoch 19/250, Loss: 0.4486
Epoch 20/250, Loss: 0.4377
Epoch 21/250, Loss: 0.4332
Epoch 22/250, Loss: 0.4303
Epoch 23/250, Loss: 0.4228
Epoch 24/250, Loss: 0.4189
Epoch 25/250, Loss: 0.4243
Epoch 26/250, Loss: 0.4197
Epoch 27/250, Loss: 0.4149
Epoch 28/250, Loss: 0.4134
Epoch 29/250, Loss: 0.4041
Epoch 30/250, Loss: 0.4010
Epoch 31/250, Loss: 0.3991
Epoch 32/250, Loss: 0.3870
Epoch 33/250, Loss: 0.3950
Epoch 34/250, Loss: 0.3924
Epoch 35/250, Loss: 0.3907
Epoch 36/250, Loss: 0.3845
Epoch 37/250, Loss: 0.3754
Epoch 38/2

In [9]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')

Accuracy on test set: 60.69%


In [10]:
# Save it
torch.save(model.state_dict(), 'emotion_model.pth')